# 프롬노웨어 크롤링
- 남녀구분 잘 안됨
- 카테고리별로 크롤링 진행
- 한 페이지당 100개 품목


In [1]:
import time
import json
from collections import defaultdict

import selenium
from selenium import webdriver
from selenium.webdriver import ActionChains

from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By

from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import Select
from selenium.webdriver.support.ui import WebDriverWait

from CreateDict import create_dict
from get_brand_list import get_brand_list
from beepsound import beepsound

In [9]:
# 1페이지에 존재하는 품목들을 가져오는 함수
def get_product_list():
    css = '#contents > div.xans-element-.xans-product.xans-product-normalpackage > div.xans-element-.xans-product.xans-product-listnormal > ul'
    xpath = '//*[starts-with(@id, "anchorBoxId_")]'
    product_list = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CSS_SELECTOR, css)))
    products = product_list.find_elements_by_xpath(xpath) # 여기에 모든 element가 담김(리스트로)
    return products

# 품목 카테고리 가져오는 함수
def get_product_category():
    """ex: 니트 · 가디건"""

    css = '#contents > div.xans-element-.xans-product.xans-product-menupackage > div > h2'
    product_category = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CSS_SELECTOR, css)))
    product_category = product_category.text
    print(product_category)
    return product_category

# 품목을 클릭하는 함수
def click_product(iter):
    css1 = '#contents > div.xans-element-.xans-product.xans-product-normalpackage > div.xans-element-.xans-product.xans-product-listnormal > ul'
    xpath = '//*[starts-with(@id, "anchorBoxId_")]'
    css2 = 'div > a'
    # 드라이버 재정의
    product_list = WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.CSS_SELECTOR, css1)))
    products = product_list.find_elements_by_xpath(xpath)
    # url
    product_link = products[iter].find_element_by_css_selector('div > a').get_attribute('href')
    # 박스 클릭
    product_box = WebDriverWait(products[iter], 10).until(EC.element_to_be_clickable((By.CSS_SELECTOR, css2)))
    product_box.click()
    print(f'link:{product_link}')
    time.sleep(1)
    return product_link

# 품목 이름을 가져오는 함수
def get_product_name():
    class_name = 'ndc_name'
    product_name = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CLASS_NAME, class_name)))
    product_name = product_name.text
    print(f'name:{product_name}')
    return product_name

# 품목 브랜드를 가져오는 함수
def get_product_brand():
    """
    명시된 브랜드명을 그대로 가져옴
    우리 브랜드명에 맞춘 변환은 get_brand_list.py 함수에서 함
    실행은 get_clothes_info() 함수에서 받아서 변환
    """
    css = '#contents > div.xans-element-.xans-product.xans-product-detail > div.detailArea > div.infoArea > div.xans-element-.xans-product.xans-product-detaildesign > table > tbody > tr:nth-child(2) > td'
    product_brand = WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.CSS_SELECTOR, css)))
    product_brand = product_brand.text
    print(f'brand:{product_brand}')
    return product_brand

# 품목 가격을 가져오는 함수
def get_product_price():
    css = '#contents > div.xans-element-.xans-product.xans-product-detail > div.detailArea > div.infoArea > div.xans-element-.xans-product.xans-product-detaildesign > table > tbody > tr:nth-child(5) > td > span'
    product_price = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CSS_SELECTOR, css)))
    product_price = product_price.text
    product_price = product_price.replace(",", '').replace(" ", "").replace("KRW", '')
    print(f'price:{product_price}')
    return product_price

# 품목 사이즈를 가져오는 함수
def get_product_size():
    id = 'prdDetail'
    product_sizes = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.ID, id))).get_attribute("innerHTML")
    product_sizes = product_sizes.strip().split('\n')
    try: # 가슴 사이즈가 있으면 가슴 사이즈만 추출
        product_size = [x for x in product_sizes if '가슴' in x][0].split('가슴')[1].split('cm')[0].strip()
        type_ = '가슴'
    except: # 가슴 사이즈 없고 허리 사이즈 있으면 허리 사이즈 추출
        product_size = [x for x in product_sizes if '허리' in x][0].split('허리')[1].split('cm')[0].strip()
        type_ = '허리'
    print(f'type: {type_}, size:{product_size}')
    return type_, product_size

# 썸네일 이미지를 가져오는 함수
def get_product_thumbnail():
    xpath = '//*[@id="contents"]/div[1]/div[1]/div[2]/div/img'
    product_thumbnail = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, xpath))).get_attribute('src')
    print(f'thumbnail:{product_thumbnail}')
    return product_thumbnail

# 성별 구분하는 함수
def get_product_sex():
    css = '#contents > div.xans-element-.xans-product.xans-product-detail > div.detailArea > div.infoArea > div.xans-element-.xans-product.xans-product-detaildesign > table > tbody > tr:nth-child(3) > td'
    product_size = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CSS_SELECTOR, css)))
    product_size = product_size.text
    if product_size[0] == 'W': # 사이즈에 WL, WM 이런 식으로 되어 있으면 여자꺼
        sex = 'women'
    else: # M, L, 이런식이면 남자꺼
        sex = 'men'
    print(f'product size:{product_size}, {sex}')
    return sex
    
# 품목 카테고리 구분하는 함수
def get_product_category_type(product_category, sizing_name):
    if product_category == '반팔 · 반바지':
        if sizing_name == '가슴':
            category = 'top'
        elif sizing_name == '허리':
            category = 'bottom'
    elif product_category == '치마 · 원피스':
        if sizing_name == '가슴':
            category = 'dress'
        elif sizing_name == '허리':
            category = 'skirt'
    elif product_category in ['후디 · 맨투맨', '셔츠 · 긴팔티', '니트 · 가디건']:
        category = 'top'
    elif product_category in ['코트 · 아우터', '점퍼 · 스포츠', '자켓 · 블루종']:
        category = 'outer'
    else: # 가끔 품목에 기타 카테고리가 있음
        category = 'others'
    print(f'category:{category}')
    return category

# 품절 여부 확인하는 함수
def is_product_sold_out():
    """sold out 아이콘이 있는지 체킹해서 품절 여부를 판단한다."""
    
    css = '#contents > div.xans-element-.xans-product.xans-product-detail > div.detailArea > div.headingArea > span.icon > img'
    try:
        product_sold_out = WebDriverWait(driver, 0.5).until(EC.presence_of_element_located((By.CSS_SELECTOR, css)))
        product_sold_out.get_attribute('src')
        print(f'품절 상품') # img src가 있으면 sold out 아이콘이 있는 거니까 품절
        return True
    except: # img src가 없으면 판매중
        print(f'판매중인 상품')
        return False

In [4]:
def get_clothes_info(store_name, sale_price):    
    """우리 json 형식에 맞게 포매팅하는 함수
    
    Keyword arguments:
    store_name: 사이트 이름(여기서는 프롬노웨어)
    sale_price: 세일 가(여기는 세일 상품이 없음 -> '0'으로 표기)
    Return: 딕셔너리
    """
    
    info_dict = {}
    # 위에서 값들 받아옴
    values = [product_name, store_name, product_category, product_gender, product_brand_after, product_price, sale_price, product_thumbnail, product_link, is_sold_out, product_size]
    # 우리 형식에 맞는 key 값 리스트 정의
    keys = ['name', 'storeName', 'category', 'gender', 'brand', 'originalPrice', 'salePrice', 'thumbnailUrl', 'contentUrl', 'isSoldOut', 'size']
    if product_type == '가슴': # 가슴 -> key: chest
        product_type_ = 'chest'
    else:
        product_type_ = 'waist' # 허리 -> key: waist
    for i in range(len(keys)):
        if keys[i] == 'size':
            info_dict[keys[i]] = {}
            info_dict[keys[i]][product_type_] = product_size 
        else:
            info_dict[keys[i]] = values[i]
    return info_dict

In [12]:
driver = webdriver.Chrome(executable_path='chromedriver')
store_name = '프롬노웨어'
sale_price = '0'
for i in range(1, 13): ##### 바꿔 #####
    URL = f'http://fromnowhere.co.kr/product/list.html?cate_no=157&page={i}' ##### 바꿔 #####
    driver.get(url=URL)
    with open(f'{store_name}.json', 'r', encoding='utf-8') as f:
        total_list = json.load(f)
    product_list = get_product_list()
    iter = 1
    for i in range(len(product_list)):
        print(f'----------------{i} page,{iter}-----------------')
        # print(product_list[i].get_attribute('innerHTML'))

        product_categories = get_product_category()
        product_link = click_product(i)
        product_name = get_product_name()
        product_brand_before = get_product_brand()
        try:
            product_price = get_product_price()
            product_type, product_size = get_product_size()
        except:
            time.sleep(0.5)
            driver.get(url=URL)
            continue
        product_thumbnail = get_product_thumbnail()
        product_gender = get_product_sex()
        product_category = get_product_category_type(product_categories, product_type)
        is_sold_out = is_product_sold_out()
        product_brand_after = get_brand_list(product_brand_before)
        json_ = get_clothes_info(store_name=store_name, sale_price=sale_price)
        total_list.append(json_)
        iter += 1
        
        time.sleep(0.5)
        driver.get(url=URL)
    with open(f'{store_name}.json', 'w', encoding='utf-8') as jsonfile:
        json.dump(total_list, jsonfile, indent=4, ensure_ascii=False)

beepsound()

----------------1-----------------
니트 · 가디건
link:http://fromnowhere.co.kr/product/detail.html?product_no=149238&cate_no=157&display_group=1
name:110 갭 코튼믹스 가디건 루즈핏
brand:GAP
price:9900
type: 가슴, size:52
thumbnail:http://fromnowhere.co.kr/web/product/big/a5083.jpg
product size:2XL, men
category:top
판매중인 상품
----------------2-----------------
니트 · 가디건
link:http://fromnowhere.co.kr/product/detail.html?product_no=149237&cate_no=157&display_group=1
name:100 울/아크릴 니트 릴렉스핏 082
brand:none
price:9900
type: 가슴, size:48
thumbnail:http://fromnowhere.co.kr/web/product/big/a5082.jpg
product size:L, men
category:top
판매중인 상품
----------------3-----------------
니트 · 가디건
link:http://fromnowhere.co.kr/product/detail.html?product_no=149236&cate_no=157&display_group=1
name:95 아크릴 니트 루즈크롭핏 081
brand:none
price:2900
type: 가슴, size:54
thumbnail:http://fromnowhere.co.kr/web/product/big/a5081.jpg
product size:M, men
category:top
판매중인 상품
----------------4-----------------
니트 · 가디건
link:http://fromnowhere.co.kr/pro

# 파일 합치기
- 노트북으로 크롤링한 거 한 파일로 합치기

In [ ]:
check = input(f'저장하시겠습니까?(y/n)')
if check == 'y':
    file1_name = '프롬노웨어.json' # 첫번째 파일명
    file2_name = '프롬노웨어2.json' # 두번째 파일명
    upload_file_name = '프롬노웨어.json' # 업로드 파일명
    list1 = json.load(file1_name, encoding='utf-8')
    list2 = json.load(file2_name, encoding='utf-8')
    to_upload = list1.extend(list2)
    with open(f'{upload_file_name}', 'w', encoding='utf-8') as jsonfile:
        json.dump(to_upload, jsonfile, indent=4, ensure_ascii=False)
    print('저장 완료')
else:
    print('생략')
